In [1]:
import pandas as pd
from ggplot import *
import numpy as np
%matplotlib inline

# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

In [2]:
df = pd.read_csv('world_war.csv')

# Quick glance at the data

In [3]:
df.head(10)

,Name,Year,Assaulter,Defender,Assaulter_1,Defender_1,Assaulter_strength,Defender_strength,Outcome
0,Baltic Operation,1941,Nazi Germany,Soviet Union,NaN,NaN,787 500,498 000,win
1,Battle of Bialystok–Minsk,1941,Nazi Germany,Soviet Union,NaN,NaN,750 000,675 000,win
2,Battle of Brody,1941,Nazi Germany,Soviet Union,NaN,NaN,750,3500,win
3,Battle of Hanko,1941,Nazi Germany,Soviet Union,Finland,NaN,25 300,30 000,loss
4,Demyansk Pocket,1942,Soviet Union,Nazi Germany,NaN,NaN,130 000,400 000,loss


In [4]:
#Create list of Assaulters
assaulter_list = []

for elem in df['Assaulter'].unique():
    if type(elem) == str:
        assaulter_list.append(elem)
    else:
        continue

#Create list of Defenders
defender_list = []

for elem in df['Defender'].unique():
    if type(elem) == str:
        defender_list.append(elem)
    else:
        continue


In [27]:
#How many times has been successfully carried out the attack
wins_at = []

for elem in assaulter_list:
    score = 0
    for row in df['Assaulter'][df['Outcome'] == 'win']:
        if elem in row:
            score +=1
        else:
            continue
    wins_at.append(score)

data = {'Assaulter': assaulter_list,
        'Win': wins_at}
attack_win = pd.DataFrame(data)

In [28]:
#How many times has been unsuccessfully carried out the attack
loss_at = []

for elem in assaulter_list:
    score = 0
    for row in df['Assaulter'][df['Outcome'] == 'loss']:
        if elem in row:
            score +=1
        else:
            continue
    loss_at.append(score)

data = {'Assaulter': assaulter_list,
        'Loss': loss_at}
attack_loss = pd.DataFrame(data)

In [32]:
#How many times has been successfully carried out protection
wins_pr = []

for elem in defender_list:
    score = 0
    for row in df['Defender'][df['Outcome'] == 'loss']:
        if elem in row:
            score +=1
        else:
            continue
    wins_pr.append(score)

data = {'Defender': defender_list,
        'Win': wins_pr}
protection_win = pd.DataFrame(data)

In [33]:
#How many times has been unsuccessfully carried out protection
loss_pr = []

for elem in defender_list:
    score = 0
    for row in df['Defender'][df['Outcome'] == 'win']:
        if elem in row:
            score +=1
        else:
            continue
    loss_pr.append(score)

data = {'Defender': defender_list,
        'Loss': loss_pr}
protection_loss = pd.DataFrame(data)

In [38]:
#Create a list of participants 
participants = []

participants = set(assaulter_list + defender_list)

#Dictionary
data = {'attack_win': wins_at,
        'attack_loss': loss_at,
        'protection_win': wins_pr,
        'protection_loss': loss_pr, }

# Create a dataframe from the dictionary, indexed by a participant's name
battle_record = pd.DataFrame(data, index = participants)
battle_record

,attack_loss,attack_win,protection_loss,protection_win
Nazi Germany,1,3,0,1
Soviet Union,1,0,3,1


In [39]:
# Count the total attacks
battle_record['total_attacks'] = battle_record['attack_win'] + battle_record['attack_loss']

# Count the total defends
battle_record['total_defends'] = battle_record['protection_win'] + battle_record['protection_loss']

# Count the total wins
battle_record['total_wins'] = battle_record['attack_win'] + battle_record['protection_win']

# Count the total losses
battle_record['total_loss'] = battle_record['attack_loss'] + battle_record['protection_loss']

# Create a total number of battles
battle_record['total_battles'] = battle_record['total_attacks'] + battle_record['total_defends']

# Create a win percentage score (total wins / total battles)
battle_record['win_percentage'] = battle_record['total_wins'] / battle_record['total_battles'] * 100

# Create a composite score (total number of wins minus total number of losses)
battle_record['composite_record'] = battle_record['total_wins'] - battle_record['total_loss']

battle_record

,attack_loss,attack_win,protection_loss,protection_win,total_attacks,total_defends,total_wins,total_loss,total_battles,win_percentage,composite_record
Nazi Germany,1,3,0,1,4,1,4,1,5,80,3
Soviet Union,1,0,3,1,1,4,1,4,5,20,-3


# Analysis

In [37]:
# Top 10 most active attackers
battle_record.sort('total_attacks', ascending=False).head(10)['total_attacks']

Nazi Germany    4
Soviet Union    1
Name: total_attacks, dtype: int64